In [11]:

import scanpy as sc
import sys
import os

sys.path.append('..')  # Add parent directory to Python path
import scanpy as sc
from omnicell.evaluation.utils import get_DEGs


In [5]:
adata = sc.read("/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/K562_essential_raw_singlecell_01.h5ad")

/orcd/data/omarabu/001/opitcho/miniforge3/envs/omnicell/lib/python3.9/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [15]:
adata.obs['cell_type']

cell_barcode
AAACCCAAGAAATCCA-27    k562
AAACCCAAGAACTTCC-31    k562
AAACCCAAGAAGCCAC-34    k562
AAACCCAAGAATAGTC-43    k562
AAACCCAAGACAGCGT-28    k562
                       ... 
TTTGTTGTCTGTCGTC-45    k562
TTTGTTGTCTGTCTCG-27    k562
TTTGTTGTCTGTGCGG-44    k562
TTTGTTGTCTTGCAGA-14    k562
TTTGTTGTCTTTACAC-25    k562
Name: cell_type, Length: 310385, dtype: category
Categories (1, object): ['k562']

In [6]:
adata.obs['gene'].unique()

['NAF1', 'BUB1', 'UBL5', 'C9orf16', 'TIMM9', ..., 'RPL7A', 'NUP155', 'FDPS', 'RBM22', 'POLR3A']
Length: 2058
Categories (2058, object): ['AAAS', 'AAMP', 'AARS', 'AARS2', ..., 'ZRSR2', 'ZW10', 'ZWINT', 'non-targeting']

In [9]:
type(adata.obs["gene"])

pandas.core.series.Series

In [12]:
adata_control = adata[adata.obs["gene"] == 'non-targeting']
adata_pert = adata[adata.obs["gene"] == 'NAF1']

In [18]:
adata.var

,gene_name,chr,start,end,class,strand,length,in_matrix,mean,std,cv,fano,gene_id
gene_name,,,,,,,,,,,,,
LINC01409,LINC01409,chr1,778747,810065,gene_version10,+,31318,True,0.137594,0.380048,2.762105,1.049733,ENSG00000237491
LINC01128,LINC01128,chr1,825138,868202,gene_version9,+,43064,True,0.256720,0.520162,2.026184,1.053944,ENSG00000228794
NOC2L,NOC2L,chr1,944203,959309,gene_version11,-,15106,True,1.975144,1.707837,0.864665,1.476706,ENSG00000188976
KLHL17,KLHL17,chr1,960584,965719,gene_version14,+,5135,True,0.119593,0.353702,2.957540,1.046089,ENSG00000187961
HES4,HES4,chr1,998962,1000172,gene_version10,-,1210,True,0.249577,0.561933,2.251540,1.265214,ENSG00000188290
...,...,...,...,...,...,...,...,...,...,...,...,...,...
BX004987.1,BX004987.1,GL000009.2,56140,58376,gene_version1,-,2236,True,0.241213,0.507266,2.102976,1.066768,ENSG00000278704
MAFIP,MAFIP,GL000194.1,53594,115055,gene_version1,-,61461,True,0.127525,0.361556,2.835168,1.025072,ENSG00000274847
AL354822.1,AL354822.1,GL000218.1,51867,54893,gene_version1,-,3026,True,0.248814,0.516552,2.076062,1.072394,ENSG00000278384


In [26]:
adata.var_names.duplicated().any()

True

In [27]:
raw_gene_dups = adata.var_names[adata.var_names.duplicated()]


In [28]:
raw_gene_dups

CategoricalIndex(['TBCE', 'HSPA14'], categories=['A1BG', 'AAAS', 'AACS', 'AAGAB', ..., 'ZYG11A', 'ZYG11B', 'ZYX', 'ZZEF1'], ordered=False, dtype='category', name='gene_name')

In [30]:

import numpy as np
duplicate_mask = adata.var_names.duplicated(keep=False)  # keep=False marks all duplicates
duplicate_indices = np.where(duplicate_mask)[0]

# Show duplicates with their positions
duplicate_series = adata.var_names[duplicate_mask]

In [39]:
print(duplicate_series)
print(duplicate_indices)

CategoricalIndex(['TBCE', 'TBCE', 'HSPA14', 'HSPA14'], categories=['A1BG', 'AAAS', 'AACS', 'AAGAB', ..., 'ZYG11A', 'ZYG11B', 'ZYX', 'ZZEF1'], ordered=False, dtype='category', name='gene_name')
[ 858  859 4168 4169]


In [34]:
dup1 = adata[:, 858]
dup2 = adata[:, 859]

In [38]:
(dup1.X == dup2.X).sum()

237434